In [1]:
from __future__ import print_function
import torch
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import torch.utils.data
import torch.nn.init as init
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import VAE

print( 'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

torch: 1.3.0+cpu cuda: False


## Load & Re-Scale Data Sets

In [2]:
path = "train_2_1000Samples.csv"
train_data = pd.read_csv(path).iloc[:,1:]
test_data = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_2_200Samples.csv')).iloc[:,1:]
train_data = train_data.iloc[:,:-1]
test_data = test_data.iloc[:,:-1]
train_data.columns  = test_data.columns
cols = test_data.columns
scalar = MinMaxScaler().fit(pd.concat([train_data, test_data]))
train_data = pd.DataFrame(scalar.transform (train_data)) 
test_data = pd.DataFrame(scalar.transform (test_data)) 
train_data.columns = cols
test_data.columns = cols

## Fix the Global Variables for Learning

In [3]:
#hyperparameters
x_dim = train_data.shape[1]
h_dim = 250
z_dim = 25
n_epochs = 40
clip = 10
learning_rate = 1e-3
batch_size = 10
seed = 100
print_every = 10
save_every = 10
#manual seed
torch.manual_seed(seed)
#init model + optimizer + datasets
train_loader = torch.utils.data.DataLoader ( dataset = train_data.values ,  batch_size = batch_size , shuffle= True)
test_loader = torch.utils.data.DataLoader (  dataset = test_data.values , shuffle= True)
model = VAE.VAE(x_dim, h_dim, z_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train and Test the VAE

In [4]:
def train(epoch):
    train_loss = 0
    epoch_loss = np.zeros(int(len (train_data) / batch_size ))
    epoch_div = np.zeros(int(len (train_data) / batch_size))
    for batch_idx, (data) in enumerate(train_loader):
        
        data = Variable(data)
        #forward + backward + optimize
        optimizer.zero_grad()
        kld_loss, nll_loss, lat, recon, _ = model(data)
        epoch_loss [batch_idx] = nll_loss
        epoch_div [batch_idx] = kld_loss
        loss = kld_loss + nll_loss
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t KLD Loss: {:.6f} \t NLL Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                kld_loss.data / batch_size,
                nll_loss.data / batch_size))

            

        train_loss += loss.data
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    return epoch_loss, epoch_div
    
def test(epoch):
    """uses test data to evaluate 
    likelihood of the model"""
    mean_kld_loss, mean_nll_loss = 0, 0
    epoch_loss = np.zeros(len(test_data))
    epoch_div = np.zeros(len(test_data))
    for i, (data) in enumerate(test_loader):                                           
        
        data = Variable(data.reshape(1,-1))
        kld_loss, nll_loss, _, _, _ = model(data)
        epoch_div [i] = kld_loss
        epoch_loss [i] = nll_loss
        mean_kld_loss += kld_loss.data
        mean_nll_loss += nll_loss.data

    mean_kld_loss /= len(test_loader.dataset)
    mean_nll_loss /= len(test_loader.dataset)

    print('====> Test set loss: KLD Loss = {:.4f}, NLL Loss = {:.4f} '.format(
        mean_kld_loss, mean_nll_loss))
    return epoch_loss, epoch_div

## Train and Test the Model

In [5]:
train_error = np.zeros([n_epochs , int(train_data.shape[0] / batch_size ) ])
train_div = np.zeros([n_epochs , int(train_data.shape[0] / batch_size ) ])
test_error , test_div  = np.zeros([n_epochs , test_data.shape[0]]) , np.zeros([n_epochs , test_data.shape[0]]) 
for epoch in range(1, n_epochs + 1):
    #training + testing
    tr = train(epoch)
    train_error [epoch-1 , :] = tr [0]
    train_div [epoch-1 , :] = tr [1] 
    te = test(epoch)
    test_error [epoch-1 , :] = te [0]
    test_div [epoch-1 , :] = te [1]

Train Epoch: 1 [0/1000 (0%)]	 KLD Loss: 9.617811 	 NLL Loss: 56.044445
Train Epoch: 1 [100/1000 (10%)]	 KLD Loss: 7.006464 	 NLL Loss: 36.980578
Train Epoch: 1 [200/1000 (20%)]	 KLD Loss: 5.451535 	 NLL Loss: 22.454949
Train Epoch: 1 [300/1000 (30%)]	 KLD Loss: 5.558869 	 NLL Loss: 10.776991
Train Epoch: 1 [400/1000 (40%)]	 KLD Loss: 3.436667 	 NLL Loss: 9.738281
Train Epoch: 1 [500/1000 (50%)]	 KLD Loss: 2.004300 	 NLL Loss: 10.339705
Train Epoch: 1 [600/1000 (60%)]	 KLD Loss: 1.817669 	 NLL Loss: 9.938670
Train Epoch: 1 [700/1000 (70%)]	 KLD Loss: 1.626092 	 NLL Loss: 6.265789
Train Epoch: 1 [800/1000 (80%)]	 KLD Loss: 1.740163 	 NLL Loss: 10.273572
Train Epoch: 1 [900/1000 (90%)]	 KLD Loss: 1.198061 	 NLL Loss: 10.017974
====> Epoch: 1 Average loss: 19.9136
====> Test set loss: KLD Loss = 1.3069, NLL Loss = 8.9413 
Train Epoch: 2 [0/1000 (0%)]	 KLD Loss: 1.330577 	 NLL Loss: 9.453414
Train Epoch: 2 [100/1000 (10%)]	 KLD Loss: 1.092769 	 NLL Loss: 8.010003
Train Epoch: 2 [200/1000 (2

Train Epoch: 20 [300/1000 (30%)]	 KLD Loss: 1.245117 	 NLL Loss: -29.102649
Train Epoch: 20 [400/1000 (40%)]	 KLD Loss: 1.431165 	 NLL Loss: -35.734675
Train Epoch: 20 [500/1000 (50%)]	 KLD Loss: 1.350266 	 NLL Loss: -31.210643
Train Epoch: 20 [600/1000 (60%)]	 KLD Loss: 1.372555 	 NLL Loss: -34.147420
Train Epoch: 20 [700/1000 (70%)]	 KLD Loss: 1.365668 	 NLL Loss: -32.173623
Train Epoch: 20 [800/1000 (80%)]	 KLD Loss: 1.363986 	 NLL Loss: -38.631681
Train Epoch: 20 [900/1000 (90%)]	 KLD Loss: 1.294793 	 NLL Loss: -35.834687
====> Epoch: 20 Average loss: -32.9350
====> Test set loss: KLD Loss = 1.3555, NLL Loss = -33.3746 
Train Epoch: 21 [0/1000 (0%)]	 KLD Loss: 1.335259 	 NLL Loss: -36.685435
Train Epoch: 21 [100/1000 (10%)]	 KLD Loss: 1.261760 	 NLL Loss: -36.265507
Train Epoch: 21 [200/1000 (20%)]	 KLD Loss: 1.268936 	 NLL Loss: -36.688162
Train Epoch: 21 [300/1000 (30%)]	 KLD Loss: 1.327939 	 NLL Loss: -35.377469
Train Epoch: 21 [400/1000 (40%)]	 KLD Loss: 1.359772 	 NLL Loss: -3

Train Epoch: 39 [400/1000 (40%)]	 KLD Loss: 1.278014 	 NLL Loss: -42.899150
Train Epoch: 39 [500/1000 (50%)]	 KLD Loss: 1.257759 	 NLL Loss: -43.483424
Train Epoch: 39 [600/1000 (60%)]	 KLD Loss: 1.184289 	 NLL Loss: -39.217758
Train Epoch: 39 [700/1000 (70%)]	 KLD Loss: 1.225090 	 NLL Loss: -42.312406
Train Epoch: 39 [800/1000 (80%)]	 KLD Loss: 1.164493 	 NLL Loss: -40.081457
Train Epoch: 39 [900/1000 (90%)]	 KLD Loss: 1.159845 	 NLL Loss: -38.418432
====> Epoch: 39 Average loss: -40.4139
====> Test set loss: KLD Loss = 1.2250, NLL Loss = -40.2917 
Train Epoch: 40 [0/1000 (0%)]	 KLD Loss: 1.181031 	 NLL Loss: -41.078247
Train Epoch: 40 [100/1000 (10%)]	 KLD Loss: 1.290032 	 NLL Loss: -43.288922
Train Epoch: 40 [200/1000 (20%)]	 KLD Loss: 1.255428 	 NLL Loss: -46.245809
Train Epoch: 40 [300/1000 (30%)]	 KLD Loss: 1.291833 	 NLL Loss: -39.323837
Train Epoch: 40 [400/1000 (40%)]	 KLD Loss: 1.233235 	 NLL Loss: -40.513603
Train Epoch: 40 [500/1000 (50%)]	 KLD Loss: 1.272018 	 NLL Loss: -3

## Get Latent Representations and Save the Model for later Use

In [6]:
train_lat = [ model (Variable(torch.tensor(train_data.iloc[idx,:].values)).reshape(1,-1))[-1] for idx in range(len(train_data)) ]
test_lat = [ model (Variable(torch.tensor(test_data.iloc[idx,:].values)).reshape(1,-1))[-1] for idx in range(len(test_data)) ]
train_lat = pd.DataFrame(torch.cat(train_lat).cpu().detach().numpy())
test_lat = pd.DataFrame(torch.cat(test_lat).cpu().detach().numpy())
cols = []
for i in range(train_lat.shape[1]):
    cols.append(str('Z'+str(i+1)))
train_lat.columns = cols
test_lat.columns = cols
train_lat.to_csv('Training_Data_Sets\\latent_50D.csv')
test_lat.to_csv('Test_Data_Sets\\latent_50D.csv')
with open('Models\\50D.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
array = np.zeros((10000,25))
for i in range(len(array)):
    array [i] = model(Variable(torch.tensor(np.zeros(50))))[-1].cpu().detach().numpy()
array1 = np.zeros((10000,25))
for i in range(len(array)):
    array1 [i] = model(Variable(torch.tensor(np.ones(50))))[-1].cpu().detach().numpy()

In [17]:
np.mean(abs(np.mean(array1, 0)-np.mean(array, 0)))/4

0.12781853826622014